In [10]:
import numpy as np
import xarray as xr
from typing import Union

In [11]:
path = r"N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250922_Paper2_Results_NCI\carbon_price\1_draw_data\xr_total_carbon.nc"
da = xr.open_dataarray(path)


In [14]:
da.sum(dim='type').to_pandas().T

scenario,carbon_low,carbon_high,carbon_low_bio_10,carbon_low_bio_20,carbon_low_bio_30,carbon_low_bio_40,carbon_low_bio_50,carbon_high_bio_10,carbon_high_bio_20,carbon_high_bio_30,...,Counterfactual_carbon_low_bio_10,Counterfactual_carbon_low_bio_20,Counterfactual_carbon_low_bio_30,Counterfactual_carbon_low_bio_40,Counterfactual_carbon_low_bio_50,Counterfactual_carbon_high_bio_10,Counterfactual_carbon_high_bio_20,Counterfactual_carbon_high_bio_30,Counterfactual_carbon_high_bio_40,Counterfactual_carbon_high_bio_50
Year,,,,,,,,,,,,,,,,,,,,,
2011,-0.110137,-0.110137,-0.034635,1.610306,1.177643,1.213956,1.711015,-0.034635,1.610306,1.177643,...,-0.144774,1.500169,1.067505,1.103817,1.600878,-0.144774,1.500169,1.067505,1.103817,1.600878
2012,-0.061831,-0.061831,0.032080,-1.925074,-1.228451,-0.424336,-0.502051,0.032080,-1.925074,-1.228451,...,-0.029752,-1.986911,-1.290284,-0.486171,-0.563884,-0.029752,-1.986911,-1.290284,-0.486171,-0.563884
2013,-0.091319,-0.091319,-4.124060,-4.591524,-3.321758,-3.131001,-1.866650,-4.124060,-4.591524,-3.321758,...,-4.215377,-4.682845,-3.413072,-3.222319,-1.957972,-4.215377,-4.682845,-3.413072,-3.222319,-1.957972
2014,-0.112204,-0.112204,-3.202314,-1.906551,-3.463570,-1.527387,-1.640108,-3.202314,-1.906551,-3.463570,...,-3.314516,-2.018748,-3.575761,-1.639582,-1.752314,-3.314516,-2.018748,-3.575761,-1.639582,-1.752314
2015,0.006085,0.006085,-5.470443,-3.869268,-2.943236,-5.956347,-13.132442,-5.470443,-3.869268,-2.943236,...,-5.464347,-3.863176,-2.937141,-5.950255,-13.126401,-5.464347,-3.863176,-2.937141,-5.950255,-13.126401
2016,0.008563,0.008563,-2.523502,-3.346486,-2.633126,-4.630021,-1.084188,-2.523502,-3.346486,-2.633126,...,-2.514936,-3.337921,-2.624565,-4.621457,-1.075626,-2.514936,-3.337921,-2.624565,-4.621457,-1.075626
2017,-0.059744,-0.059744,-4.989779,-5.179601,-8.632217,-9.894744,-2.380899,-4.989779,-5.179601,-8.632217,...,-5.049515,-5.239378,-8.691952,-9.954518,-2.440627,-5.049515,-5.239378,-8.691952,-9.954518,-2.440627
2018,-0.060370,-0.060370,-5.978766,-2.200339,-4.430007,-0.323025,-0.181712,-5.978766,-2.200339,-4.430007,...,-6.039123,-2.260705,-4.490385,-0.383389,-0.242086,-6.039123,-2.260705,-4.490385,-0.383389,-0.242086
2019,0.122578,0.122578,-5.436055,-9.111387,-2.271851,8.398472,6.504883,-5.436055,-9.111387,-2.271851,...,-5.313472,-8.988808,-2.149262,8.521058,6.627457,-5.313472,-8.988808,-2.149262,8.521058,6.627457


In [9]:
da

<xarray.DataArray 'data' (cell: 24987, lu: 28, lm: 3)> Size: 8MB
array([[[0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)
Coordinates:
  * cell     (cell) int32 100kB 0 1 2 3 4 5 ... 24982 24983 24984 24985 24986
  * lu       (lu) <U27 3kB 'Apples' 'Beef - modified land' ... 'Winter oilseeds'
  * lm       (lm) <U3 36B 'ALL' 'dry' 'irr'

In [17]:
da.sel({'type': 'HIR - Beef'}).to_pandas().T

scenario,Run_13_GHG_off_BIO_off_CUT_50,Run_12_GHG_low_BIO_off_CUT_50,Run_06_GHG_high_BIO_off_CUT_50,Run_11_GHG_low_BIO_high_CUT_10,Run_10_GHG_low_BIO_high_CUT_20,Run_09_GHG_low_BIO_high_CUT_30,Run_08_GHG_low_BIO_high_CUT_40,Run_07_GHG_low_BIO_high_CUT_50,Run_05_GHG_high_BIO_high_CUT_10,Run_04_GHG_high_BIO_high_CUT_20,Run_03_GHG_high_BIO_high_CUT_30,Run_02_GHG_high_BIO_high_CUT_40,Run_01_GHG_high_BIO_high_CUT_50
Year,,,,,,,,,,,,,
2010,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
2011,0.167307,0.152182,0.152182,2.655173e-02,0.045809,0.003718,0.004475,0.004475,2.655173e-02,0.045809,0.003718,0.004475,0.004475
2012,1.018032,1.010219,1.010219,2.875857e-01,0.200275,0.254965,0.316365,0.154933,2.875857e-01,0.200275,0.254965,0.316365,0.154933
2013,1.702844,1.698097,1.698097,2.014042e-01,0.187180,0.243630,0.099907,0.118192,2.014042e-01,0.187180,0.243630,0.099907,0.118192
2014,1.363161,1.355254,1.355254,6.877478e-02,0.044106,0.039031,0.005623,0.004478,6.877478e-02,0.044106,0.039031,0.005623,0.004478
2015,1.726048,1.728260,1.728260,7.129379e-02,0.030666,0.026241,0.006863,0.004476,7.129379e-02,0.030666,0.026241,0.006863,0.004476
2016,1.758841,1.765795,1.765795,1.064183e-01,0.017932,0.020064,0.005978,0.004474,1.064183e-01,0.017932,0.020064,0.005978,0.004474
2017,1.844984,1.837678,1.837678,3.713497e-02,0.009230,0.005996,0.004474,0.004474,3.713497e-02,0.009230,0.005996,0.004474,0.004474
2018,1.845736,1.834362,1.834362,3.622108e-02,0.001511,0.003718,0.004473,0.004473,3.622108e-02,0.001511,0.003718,0.004473,0.004473


In [18]:
da.sel({'type': 'HIR - Sheep'}).to_pandas().T

scenario,Run_13_GHG_off_BIO_off_CUT_50,Run_12_GHG_low_BIO_off_CUT_50,Run_06_GHG_high_BIO_off_CUT_50,Run_11_GHG_low_BIO_high_CUT_10,Run_10_GHG_low_BIO_high_CUT_20,Run_09_GHG_low_BIO_high_CUT_30,Run_08_GHG_low_BIO_high_CUT_40,Run_07_GHG_low_BIO_high_CUT_50,Run_05_GHG_high_BIO_high_CUT_10,Run_04_GHG_high_BIO_high_CUT_20,Run_03_GHG_high_BIO_high_CUT_30,Run_02_GHG_high_BIO_high_CUT_40,Run_01_GHG_high_BIO_high_CUT_50
Year,,,,,,,,,,,,,
2010,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
2011,1.086930e-06,7.214418e-07,7.214418e-07,2.300491e-07,0.001553,0.001553,0.001554,0.002329,2.300491e-07,0.001553,0.001553,0.001554,0.002329
2012,3.763257e-06,5.829705e-06,5.829705e-06,4.085786e-07,0.001554,0.001554,0.001556,0.002332,4.085786e-07,0.001554,0.001554,0.001556,0.002332
2013,4.539593e-06,4.723003e-06,4.723003e-06,1.553424e-03,0.001554,0.001554,0.001554,0.002329,1.553424e-03,0.001554,0.001554,0.001554,0.002329
2014,3.936732e-06,2.267236e-06,2.267236e-06,1.553272e-03,0.001553,0.001554,0.001553,0.002329,1.553272e-03,0.001553,0.001554,0.001553,0.002329
2015,5.298955e-06,1.886162e-06,1.886162e-06,1.553468e-03,0.001553,0.001553,0.001553,0.002329,1.553468e-03,0.001553,0.001553,0.001553,0.002329
2016,2.665518e-06,1.700490e-06,1.700490e-06,1.553277e-03,0.001553,0.001553,0.001553,0.002329,1.553277e-03,0.001553,0.001553,0.001553,0.002329
2017,4.982153e-06,1.008065e-06,1.008065e-06,1.553275e-03,0.001553,0.001553,0.001553,0.002328,1.553275e-03,0.001553,0.001553,0.001553,0.002328
2018,3.083869e-06,9.618025e-07,9.618025e-07,1.553137e-03,0.001553,0.001553,0.001553,0.002328,1.553137e-03,0.001553,0.001553,0.001553,0.002328


In [10]:
path = r'N:\LUF-Modelling\LUTO2_XH\LUTO2\output\20250922_Paper2_Results_NCI\carbon_price\0_base_data\Run_12_GHG_low_BIO_off_CUT_50\2050\xr_biodiversity_GBF2_priority_ag_2050.nc'
da = xr.open_dataarray(path)
da.sum().values

array(1.4942842e+08, dtype=float32)

In [12]:
da['lu'].values

array(['Apples', 'Beef - modified land', 'Beef - natural land', 'Citrus',
       'Cotton', 'Dairy - modified land', 'Dairy - natural land',
       'Grapes', 'Hay', 'Nuts', 'Other non-cereal crops', 'Pears',
       'Plantation fruit', 'Rice', 'Sheep - modified land',
       'Sheep - natural land', 'Stone fruit', 'Sugar', 'Summer cereals',
       'Summer legumes', 'Summer oilseeds', 'Tropical stone fruit',
       'Unallocated - modified land', 'Unallocated - natural land',
       'Vegetables', 'Winter cereals', 'Winter legumes',
       'Winter oilseeds'], dtype='<U27')